## Heart Disease Risk Prediction: Logistic Regression 
- By: Juan Jose Mejia Celis
## Step 1: Load and prepare the Dataset

Heart disease is a major cause of death globally. Machine learning models can help identify at-risk patients by analyzing clinical features. This notebook uses logistic regression on the Kaggle Heart Disease Dataset to predict disease presence.

First, we prepare the data and do some exploratory analysis.

## Import required libraries 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)

## Load the Dataset

In [ ]:
data = pd.read_csv("heart.csv")
data.head()

## Dataset Overview

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

## Target Variable Analysis 

In [ ]:
data['target'].value_counts()

In [ ]:
data['target'].value_counts(normalize=True)

In [ ]:
plt.figure()
data['target'].value_counts().plot(kind='bar')
plt.title("Class Distribution (Heart Disease Presence)")
plt.xlabel("Target")
plt.ylabel("Count")
plt.show()

## Missing Values Check

In [ ]:
data.isnull().sum()

## Feature Selection 

In [ ]:
features = ['age', 'chol', 'trestbps', 'thalach', 'oldpeak', 'ca']
X = data[features].values
y = data['target'].values

## Feature Normalization

In [ ]:
mean_X = X.mean(axis=0)
std_X = X.std(axis=0)

X_norm = (X - mean_X) / std_X

Logistic regression converges faster when features are normalized.

## Train Test Split 70/30 

In [ ]:
total_samples = len(y)
indices = np.arange(total_samples)
np.random.shuffle(indices)

train_size = int(0.7 * total_samples)

train_idx = indices[:train_size]
test_idx = indices[train_size:]

X_train = X_norm[train_idx]
y_train = y[train_idx]

X_test = X_norm[test_idx]
y_test = y[test_idx]

Verification of rates.

In [ ]:
print("Train disease rate:", y_train.mean())
print("Test disease rate:", y_test.mean())

Similar rates in both sets indicate a balanced split.

### Step 1 Summary

- Downloaded dataset from Kaggle (303 patient records)
- Target: 1=disease present, 0=absent
- Picked 6 features: age, cholesterol, BP, max HR, ST depression, vessels
- No missing values found
- Normalized features (helps gradient descent converge faster)
- Split: 70% train, 30% test

Ready to build the model.

# Step 2: Implement Basic Logistic Regression
## Goal
Build logistic regression from scratch to predict heart disease risk. Using gradient descent, no sklearn.

## Sigmoid Function

In [ ]:
import numpy as np

def sigmoid(z):
    """
    Sigmoid activation function
    """
    return 1 / (1 + np.exp(-z))

Output is a probability (0 to 1) representing disease risk.

## Binary Cross-Entropy Loss

Using binary cross-entropy as the loss function.

In [ ]:
def compute_cost(X, y, weights, bias):
    """
    Binary cross-entropy loss
    """
    m = X.shape[0]
    z = X @ weights + bias
    predictions = sigmoid(z)
    
    epsilon = 1e-8  # numerical stability
    cost = -(1/m) * np.sum(
        y * np.log(predictions + epsilon) +
        (1 - y) * np.log(1 - predictions + epsilon)
    )
    return cost

Penalizes wrong predictions, especially false negatives (missing sick patients).

## Gradient Computation

In [ ]:
def compute_gradients(X, y, weights, bias):
    """
    Compute gradients of cost w.r.t w and b
    """
    m = X.shape[0]
    predictions = sigmoid(X @ weights + bias)
    
    grad_w = (1/m) * (X.T @ (predictions - y))
    grad_b = (1/m) * np.sum(predictions - y)
    
    return grad_w, grad_b

## Gradient Descent Training Loop

In [ ]:
def gradient_descent(X, y, learning_rate=0.01, iterations=2000):
    """
    Train logistic regression using gradient descent
    """
    m, n_features = X.shape
    weights = np.zeros(n_features)
    bias = 0.0
    
    cost_history = []
    
    for i in range(iterations):
        grad_w, grad_b = compute_gradients(X, y, weights, bias)
        
        weights -= learning_rate * grad_w
        bias -= learning_rate * grad_b
        
        if i % 50 == 0:
            cost = compute_cost(X, y, weights, bias)
            cost_history.append(cost)
    
    return weights, bias, cost_history

Alpha=0.01 works well. Running 1000-3000 iterations ensures convergence.

## Training the Model

In [ ]:
weights, bias, cost_history = gradient_descent(
    X_train, y_train,
    learning_rate=0.01,
    iterations=2500
)

print("Final weights:", weights)
print("Final bias:", bias)

## Convergence Visualization

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
plt.plot(cost_history)
plt.xlabel("Iterations (x50)")
plt.ylabel("Cost (Binary Cross-Entropy)")
plt.title("Training Loss Convergence")
plt.grid(True)
plt.show()

Cost drops smoothly - GD is working. No oscillation means learning rate is good.

## Prediction Function

In [ ]:
def predict(X, weights, bias, threshold=0.5):
    """
    Predict class labels using learned parameters
    """
    probabilities = sigmoid(X @ weights + bias)
    return (probabilities >= threshold).astype(int)

## Model Evaluation

In [ ]:
def classification_metrics(y_true, y_pred):
    accuracy = np.mean(y_true == y_pred)
    
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    
    return accuracy, precision, recall, f1

In [ ]:
y_train_pred = predict(X_train, weights, bias)
y_test_pred = predict(X_test, weights, bias)

train_metrics = classification_metrics(y_train, y_train_pred)
test_metrics = classification_metrics(y_test, y_test_pred)

print("Train metrics (Acc, Prec, Recall, F1):", train_metrics)
print("Test metrics  (Acc, Prec, Recall, F1):", test_metrics)

## Interpretation

## Metrics Explanation

- Accuracy: overall correctness
- Recall: how many sick patients we catch (most important for medical diagnosis)
- Precision: when we predict disease, how often are we right
- F1: balance between precision and recall

Missing a sick patient (false negative) is worse than a false alarm, so recall matters more here.

# Step 3: Visualize Decision Boundaries

## Why visualize?

Logistic regression draws a straight line (boundary) between classes. By plotting pairs of features, we can see:

- Which features separate healthy vs sick patients
- If a linear boundary makes sense
- Where the model might struggle

This helps understand what the model is doing.

## Helper Function: Train 2D Logistic Regression

In [ ]:
def train_logistic_2d(X, y, alpha=0.01, iters=2000):
    """
    Train logistic regression on 2D feature space
    """
    w = np.zeros(2)
    b = 0.0
    
    for _ in range(iters):
        y_hat = sigmoid(X @ w + b)
        dw = (1 / len(y)) * (X.T @ (y_hat - y))
        db = (1 / len(y)) * np.sum(y_hat - y)
        
        w -= alpha * dw
        b -= alpha * db
        
    return w, b

## Helper Function: Plot Decision Boundary

In [ ]:
def plot_decision_boundary(X, y, w, b, feature_names):
    """
    Plot data points and logistic regression decision boundary
    """
    plt.figure(figsize=(6,5))
    
    # Scatter plot
    plt.scatter(X[y==0, 0], X[y==0, 1], label="No Disease", alpha=0.7)
    plt.scatter(X[y==1, 0], X[y==1, 1], label="Disease", alpha=0.7)
    
    # Decision boundary: w1*x1 + w2*x2 + b = 0
    x_vals = np.linspace(X[:,0].min(), X[:,0].max(), 100)
    y_vals = -(w[0] * x_vals + b) / w[1]
    
    plt.plot(x_vals, y_vals, color="black", linestyle="--", label="Decision Boundary")
    
    plt.xlabel(feature_names[0])
    plt.ylabel(feature_names[1])
    plt.legend()
    plt.grid(True)
    plt.title(f"Decision Boundary: {feature_names[0]} vs {feature_names[1]}")
    plt.show()

## Feature Pair 1: Age vs Cholesterol
Medical context

- Age is a strong demographic risk factor

- Cholesterol is directly linked to atherosclerosis

In [ ]:
features_1 = ["age", "chol"]
X_pair = data[features_1].values
y_pair = data["target"].values

# Normalize
X_pair = (X_pair - X_pair.mean(axis=0)) / X_pair.std(axis=0)

w_1, b_1 = train_logistic_2d(X_pair, y_pair)

plot_decision_boundary(X_pair, y_pair, w_1, b_1, features_1)

### Observations:
- Lots of overlap between classes
- Higher age + cholesterol correlates with disease
- Linear boundary is limited here

## Feature Pair 2: Resting BP vs Max Heart Rate

In [ ]:
features_2 = ["trestbps", "thalach"]
X_pair = data[features_2].values
y_pair = data["target"].values

X_pair = (X_pair - X_pair.mean(axis=0)) / X_pair.std(axis=0)

w_2, b_2 = train_logistic_2d(X_pair, y_pair)

plot_decision_boundary(X_pair, y_pair, w_2, b_2, features_2)

### Observations:
- Cleaner separation than age-cholesterol
- Low max HR + high BP = higher risk
- Linear model fits reasonably well

## Feature Pair 3: ST Depression vs Number of Vessels
Medical context

- ST depression (oldpeak) indicates exercise-induced ischemia

- Number of vessels (ca) shows structural severity

In [ ]:
features_3 = ["oldpeak", "ca"]
X_pair = data[features_3].values
y_pair = data["target"].values

X_pair = (X_pair - X_pair.mean(axis=0)) / X_pair.std(axis=0)

w_3, b_3 = train_logistic_2d(X_pair, y_pair)

plot_decision_boundary(X_pair, y_pair, w_3, b_3, features_3)

### Observations:
- Best separation so far
- More ST depression + vessels = disease
- These features are highly informative

## Takeaway
Some feature pairs (ST depression + vessels) separate well. Others (age + cholesterol) have too much overlap. Logistic regression works for linear patterns but can't capture complex/nonlinear relationships. For better performance, could try feature engineering or more complex models.

# Step 4: Repeat with Regularization
## Why?

Small datasets (303 patients) with correlated features can cause overfitting. Regularization helps by:

- Preventing model from being too complex
- Making weights smaller/more stable
- Better performance on new patients

## Regularized Cost Function

In [ ]:
def compute_cost_reg(X, y, w, b, lam):
    """
    Binary cross-entropy with L2 regularization
    """
    m = X.shape[0]
    y_hat = sigmoid(X @ w + b)
    
    epsilon = 1e-8
    base_cost = -(1/m) * np.sum(
        y * np.log(y_hat + epsilon) +
        (1 - y) * np.log(1 - y_hat + epsilon)
    )
    
    reg_term = (lam / (2*m)) * np.sum(w**2)
    
    return base_cost + reg_term

## Regularized Gradients

In [ ]:
def compute_gradients_reg(X, y, w, b, lam):
    """
    Gradients with L2 regularization
    """
    m = X.shape[0]
    y_hat = sigmoid(X @ w + b)
    
    dw = (1/m) * (X.T @ (y_hat - y)) + (lam/m) * w
    db = (1/m) * np.sum(y_hat - y)
    
    return dw, db

## Gradient Descent with Regularization

In [ ]:
def gradient_descent_reg(X, y, alpha=0.01, iters=2000, lam=0.0):
    """
    Logistic regression with L2 regularization
    """
    m, n = X.shape
    w = np.zeros(n)
    b = 0.0
    cost_history = []
    
    for i in range(iters):
        dw, db = compute_gradients_reg(X, y, w, b, lam)
        
        w -= alpha * dw
        b -= alpha * db
        
        if i % 50 == 0:
            cost = compute_cost_reg(X, y, w, b, lam)
            cost_history.append(cost)
    
    return w, b, cost_history

## Hyperparameter Experiment: Different λ Values

In [ ]:
lambdas = [0, 0.001, 0.01, 0.1, 1]
results = []

for lam in lambdas:
    w_reg, b_reg, _ = gradient_descent_reg(
        X_train, y_train,
        alpha=0.01,
        iters=2500,
        lam=lam
    )
    
    y_test_pred = predict(X_test, w_reg, b_reg)
    acc, prec, rec, f1 = classification_metrics(y_test, y_test_pred)
    
    weight_norm = np.linalg.norm(w_reg)
    
    results.append([lam, acc, prec, rec, f1, weight_norm])

## Regularization Results Table

In [ ]:
import pandas as pd

results_df = pd.DataFrame(
    results,
    columns=["Lambda", "Accuracy", "Precision", "Recall", "F1", "||w||"]
)

results_df

## Cost Comparison Plot

In [ ]:
plt.figure(figsize=(6,4))

for lam in [0, 0.01, 0.1]:
    _, _, cost_hist = gradient_descent_reg(
        X_train, y_train,
        alpha=0.01,
        iters=2000,
        lam=lam
    )
    plt.plot(cost_hist, label=f"λ = {lam}")

plt.xlabel("Iterations (x50)")
plt.ylabel("Cost")
plt.title("Effect of L2 Regularization on Convergence")
plt.legend()
plt.grid(True)
plt.show()

## Decision Boundary: Regularized vs Unregularized

In [ ]:
features = ["oldpeak", "ca"]
X_pair = data[features].values
y_pair = data["target"].values

X_pair = (X_pair - X_pair.mean(axis=0)) / X_pair.std(axis=0)

# Unregularized
w0, b0 = train_logistic_2d(X_pair, y_pair)

# Regularized
wR, bR, _ = gradient_descent_reg(X_pair, y_pair, lam=0.1)

plt.figure(figsize=(6,5))
plt.scatter(X_pair[y_pair==0,0], X_pair[y_pair==0,1], label="No Disease", alpha=0.6)
plt.scatter(X_pair[y_pair==1,0], X_pair[y_pair==1,1], label="Disease", alpha=0.6)

x_vals = np.linspace(X_pair[:,0].min(), X_pair[:,0].max(), 100)
plt.plot(x_vals, -(w0[0]*x_vals + b0)/w0[1], "--", label="Unregularized")
plt.plot(x_vals, -(wR[0]*x_vals + bR)/wR[1], label="Regularized (λ=0.1)")

plt.xlabel("ST Depression")
plt.ylabel("Number of Vessels")
plt.legend()
plt.grid(True)
plt.title("Effect of Regularization on Decision Boundary")
plt.show()

## Results

- Higher λ → smaller weights
- Boundaries become smoother
- Moderate λ (0.01-0.1) works best
- Too much λ → underfitting (model too simple)

Best value seems to be λ around 0.01-0.1.

# Step 5: Explore Deployment in Amazon SageMaker
## Why deploy?

Training is just the first step. For real-world use, models need to:

- Run in the cloud
- Be accessible via API
- Handle real-time requests

SageMaker provides infrastructure for deploying ML models in production.

## Exporting the Trained Model

In [ ]:
# Save best model parameters
model_params = {
    "weights": w_reg,
    "bias": b_reg,
    "lambda": 0.1
}

np.save("heart_lr_model.npy", model_params)

## Inference Function

In [ ]:
def predict_patient(features, w, b):
    """
    Predict heart disease risk for a single patient
    """
    z = np.dot(features, w) + b
    probability = sigmoid(z)
    return probability

## Example Inference

In [ ]:
# Example patient (normalized with training stats)
sample_patient = np.array([60, 300, 140, 150, 2.3, 2])

sample_patient = (sample_patient - X_train.mean(axis=0)) / X_train.std(axis=0)

risk = predict_patient(sample_patient, w_reg, b_reg)

print(f"Predicted heart disease risk: {risk:.2f}")

### Notes:
- Output is between 0 and 1
- Above 0.5 = high risk
- Could be used in a clinical decision system

## SageMaker Execution

Process:

1. Opened SageMaker Studio
2. Created notebook instance (Python 3)
3. Uploaded heart_disease_lr_analysis.ipynb and heart.csv
4. Ran all cells
5. Checked plots and metrics

## Sample Endpoint Output
Example request

In [ ]:
Age=60, Chol=300, BP=140, MaxHR=150, STDepression=2.3, Vessels=2

Model response

In [ ]:
Predicted Probability = 0.68
Risk Level: High

## Comparison: Local vs SageMaker Execution
| Aspect               | Local Execution | SageMaker Execution |
| -------------------- | --------------- | ------------------- |
| Environment          | Laptop          | Managed cloud       |
| Reproducibility      | Manual          | High                |
| Scalability          | Limited         | High                |
| Cost                 | Free            | Free tier           |
| Enterprise readiness | Low             | High                |


- Observation
Results and plots were identical, validating environment consistency.

## Final Thoughts

This lab shows:

- How to build logistic regression from scratch
- Working with medical data
- Deploying on cloud platforms
- Even simple models can be useful in practice